In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F

In [4]:
spark = SparkSession.builder.master("local[4]").appName("1484").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 01:04:32 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/24 01:04:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/24 01:04:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
'''
Table Activities:

+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| sell_date   | date    |
| product     | varchar |
+-------------+---------+
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each row of this table contains the product name and the date it was sold in a market.
 

Write a solution to find for each date the number of different products sold and their names.

The sold products names for each date should be sorted lexicographically.

Return the result table ordered by sell_date.

The result format is in the following example.

 

Example 1:

Input: 
Activities table:
+------------+------------+
| sell_date  | product     |
+------------+------------+
| 2020-05-30 | Headphone  |
| 2020-06-01 | Pencil     |
| 2020-06-02 | Mask       |
| 2020-05-30 | Basketball |
| 2020-06-01 | Bible      |
| 2020-06-02 | Mask       |
| 2020-05-30 | T-Shirt    |
+------------+------------+
Output: 
+------------+----------+------------------------------+
| sell_date  | num_sold | products                     |
+------------+----------+------------------------------+
| 2020-05-30 | 3        | Basketball,Headphone,T-shirt |
| 2020-06-01 | 2        | Bible,Pencil                 |
| 2020-06-02 | 1        | Mask                         |
+------------+----------+------------------------------+
Explanation: 
For 2020-05-30, Sold items were (Headphone, Basketball, T-shirt), 
we sort them lexicographically and separate them by a comma.
For 2020-06-01, Sold items were (Pencil, Bible), 
we sort them lexicographically and separate them by a comma.
For 2020-06-02, the Sold item is (Mask), we just return it.
'''

In [5]:
data = [
('2020-05-30','Headphone' ),
('2020-06-01','Pencil'    ),
('2020-06-02','Mask'      ),
('2020-05-30','Basketball'),
('2020-06-01','Bible'     ),
('2020-06-02','Mask'      ),
('2020-05-30','T-Shirt'   )  
]
schema = ['sell_date','product']

In [6]:
df = spark.createDataFrame(data=data, schema=schema)
df.show()

+----------+----------+
| sell_date|   product|
+----------+----------+
|2020-05-30| Headphone|
|2020-06-01|    Pencil|
|2020-06-02|      Mask|
|2020-05-30|Basketball|
|2020-06-01|     Bible|
|2020-06-02|      Mask|
|2020-05-30|   T-Shirt|
+----------+----------+



In [12]:
df.groupBy(F.col("sell_date"))\
    .agg(
            F.countDistinct(F.col("product")).alias("num_sold")
        )\
    .show()

[Stage 8:>                                                          (0 + 4) / 4]

+----------+--------+
| sell_date|num_sold|
+----------+--------+
|2020-05-30|       3|
|2020-06-01|       2|
|2020-06-02|       1|
+----------+--------+



## Complete Solution

In [14]:
df.groupBy(F.col("sell_date"))\
    .agg(
            F.countDistinct(F.col("product")).alias("num_sold"),
            F.concat_ws(",", F.sort_array(F.collect_set(F.col("product")))).alias("products")
        )\
    .show(truncate=False)

+----------+--------+----------------------------+
|sell_date |num_sold|products                    |
+----------+--------+----------------------------+
|2020-05-30|3       |Basketball,Headphone,T-Shirt|
|2020-06-01|2       |Bible,Pencil                |
|2020-06-02|1       |Mask                        |
+----------+--------+----------------------------+



## Prtformance POV

In [18]:
df.groupBy(F.col("sell_date"))\
    .agg(
            F.countDistinct(F.col("product")).alias("num_sold"),
            F.concat_ws(",", F.sort_array(F.collect_set(F.col("product")))).alias("products")
        )\
    .show(truncate=False)

+----------+--------+----------------------------+
|sell_date |num_sold|products                    |
+----------+--------+----------------------------+
|2020-05-30|3       |Basketball,Headphone,T-Shirt|
|2020-06-01|2       |Bible,Pencil                |
|2020-06-02|1       |Mask                        |
+----------+--------+----------------------------+



## SQL Solution 

<pre>

SELECT  sell_date, count(DISTINCT product) as num_sold, GROUP_CONCAT(DISTINCT product order by product) as product 
FROM Activities 
GROUP BY sell_date
    
</pre>